# # ファクトチェック機能 性能評価ノートブック
# 
# このノートブックでは、`services/factcheck.py`に実装された`factcheck_slide`関数の性能を評価します。
# - **評価指標**: 適合率(Precision), 再現率(Recall), F1スコア
# - **評価データ**: `data/ground_truth/*.json` に格納された正解データ

# ### 1. 必要なライブラリのインポートと設定
# 必要なライブラリのインポート
# バックエンドのソースコードをPythonパスに追加
# 評価に必要な定数の設定

In [ ]:
import json
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, confusion_matrix, ConfusionMatrixDisplay

# backendのソースコードをインポートするためにパスを追加
sys.path.append(str(Path.cwd().parent / 'backend' / 'extraction_service'))

from app.services.factcheck import factcheck_slide
from app.services.vector_store import VectorStore # 全チャンク取得のために追加
from app.models import Inconsistency

# --- 定数設定 ---
GROUND_TRUTH_DIR = Path.cwd().parent / 'data' / 'ground_truth'
# 評価対象のスライドIDリスト（実際のファイル名に合わせてください）
SLIDE_IDS_TO_EVALUATE = ["slide-001", "slide-002"] 

# ### 2. 評価データの読み込みと前処理
# 正解データ（ground truth）をJSONファイルから読み込む
# "矛盾"とラベル付けされたchunk_idのセットを返す

In [ ]:
def load_ground_truth(slide_id: str) -> set[str]:
    """正解データから「矛盾」とラベル付けされたchunk_idのセットを読み込む"""
    gt_path = GROUND_TRUTH_DIR / f"{slide_id}.json"
    if not gt_path.exists():
        return set()
    
    with open(gt_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        # "verdict"が"contradiction"である項目のchunk_idをセットとして返す
        return {item['chunk_id'] for item in data if item.get('verdict') == 'contradiction'}


# ### 3. 評価の実行
# 各スライドに対して`factcheck_slide`関数を実行し、予測結果と正解データを比較します。
# 各スライドに対して：
# 全チャンクIDを取得
# 正解データを読み込み
# ファクトチェック機能で予測を実行
# 正解と予測のラベルを作成

In [ ]:
vector_store = VectorStore()
y_true = [] # 正解ラベル (1: 矛盾あり, 0: 矛盾なし)
y_pred = [] # 予測ラベル (1: 矛盾あり, 0: 矛盾なし)

for slide_id in SLIDE_IDS_TO_EVALUATE:
    print(f"--- Processing slide: {slide_id} ---")
    
    # 1. スライド内の全チャンクIDを取得
    all_chunks = vector_store.get_chunks_by_document_id(slide_id)
    all_chunk_ids = {chunk.chunk_id for chunk in all_chunks}
    if not all_chunk_ids:
        print(f"No chunks found for {slide_id}, skipping.")
        continue

    # 2. 正解データを読み込み
    true_contradiction_chunks = load_ground_truth(slide_id)
    print(f"Ground Truth contradictions: {len(true_contradiction_chunks)} chunks")

    # 3. `factcheck_slide`関数で予測を実行
    predicted_inconsistencies = factcheck_slide(slide_id)
    predicted_contradiction_chunks = {
        item.chunk_id for item in predicted_inconsistencies if item.verdict == 'contradiction'
    }
    print(f"Predicted contradictions: {len(predicted_contradiction_chunks)} chunks")

    # 4. 全チャンクIDをループし、正解と予測のラベルを作成
    for chunk_id in all_chunk_ids:
        y_true.append(1 if chunk_id in true_contradiction_chunks else 0)
        y_pred.append(1 if chunk_id in predicted_contradiction_chunks else 0)

# ### 4. 評価指標の計算と表示
# 以下の指標を計算：
# Precision（適合率）
# Recall（再現率）
# F1スコア
# 結果をPandas DataFrameで表示

In [ ]:
if not y_true:
    print("Evaluation could not be performed. No data available.")
else:
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_true, 
        y_pred, 
        average='binary', # 「矛盾あり(1)」クラスに対する指標を計算
        pos_label=1,
        zero_division=0
    )

    print("\n--- Overall Performance Metrics ---")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")
    print(f"F1-Score:  {f1:.4f}")
    print("---------------------------------")
    
    # Pandas DataFrameで見やすく表示
    results_df = pd.DataFrame({
        'Metric': ['Precision', 'Recall', 'F1-Score'],
        'Score': [precision, recall, f1]
    })
    display(results_df)

### 5. 結果の可視化 (混同行列)
# 混同行列（Confusion Matrix）を生成
# matplotlib/seabornを使用して視覚化
# 矛盾の検出精度を分かりやすく表示

In [ ]:
if y_true:
    cm = confusion_matrix(y_true, y_pred, labels=[1, 0])
    disp = ConfusionMatrixDisplay(
        confusion_matrix=cm, 
        display_labels=['Contradiction', 'Not Contradiction']
    )

    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(ax=ax, cmap=plt.cm.Blues)
    ax.set_title("Fact-Check Confusion Matrix")
    plt.show()